# Настройка
Все зависимости перечислены в ячейке ниже. Кроме того, есть ещё дополнительные данные (opencorpora, например). Они тоже скачиваются в первых ячейках.

In [ ]:
%%writefile requirements.txt
nltk>=3.4.5
razdel>=0.4.0
rusenttokenize>=0.0.5
b-labs-models>=2017.8.22
lxml>=4.2.1
spacy>=2.1.4
pymystem3>=0.2.0
rnnmorph>=0.4.0

In [ ]:
import sys
!pip install --user --upgrade --force-reinstall -r requirements.txt

In [ ]:
# Restart kernel

import os
os._exit(0)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
import sys
!python -m spacy download en_core_web_sm

### Дополнительные данные

Opencorpora: 31 Мб по сети, 530 Мб в распакованном виде

In [ ]:
!wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2

In [ ]:
!bzip2 -d annot.opcorpora.xml.bz2

### Тестовые примеры

In [ ]:
example1 = "this's a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it’s your turn."
example2 = """
    An ambitious campus expansion plan was proposed by Fr. Vernon F. Gallagher in 1952.
    Assumption Hall, the first student dormitory, was opened in 1954,
    and Rockwell Hall was dedicated in November 1958, housing the schools of business and law.
    It was during the tenure of F. Henry J. McAnulty that Fr. Gallagher's ambitious plans were put to action.
"""
example3 = """
    А что насчёт русского языка? Хорошо ли сегментируются имена?
    Ай да А.С. Пушкин! Ай да сукин сын!
    «Как же так?! Захар...» — воскликнут Пронин.
    - "Так в чем же дело?" - "Не ра-ду-ют".
    И т. д. и т. п. В общем, вся газета.
    Православие... более всего подходит на роль такой идеи...
    Нефть за $27/барр. не снится.
"""
example4 = """
    Кружка-термос на 0.5л (50/64 см³, 516;...) стоит $3.88
"""
example5 = """
    Good muffins cost $3.88 in New York.  Please buy me two of them. Thanks.
"""

# Сегментация предложений
Первая задача - разбиение текста на предложения

### Экперименты

##### NLTK - Natural Language Toolkit
Популярная платформа для анализа текстов. Особенно хорошо работает для английского. В основном не содержит ничего из машинного обучения, только старые добрые правила.

In [ ]:
from nltk.tokenize import sent_tokenize
sent_tokenize(example1)

А вот тут что-то пошло не так

In [ ]:
sent_tokenize(example2)

А что насчёт русского языка?

In [ ]:
sent_tokenize(example3)

https://github.com/Mottl/ru_punkt

Data for sentence tokenization was taken from 3 sources:

  * Articles from Russian Wikipedia (about 1 million sentences)
  * Common Russian abbreviations from Russian orthographic dictionary, edited by V. V. Lopatin;
  * Generated names initials.

In [ ]:
sent_tokenize(example3, language="russian")

https://github.com/natasha/razdel

razdel старается разбивать текст на предложения и токены так, как это сделано в 4 датасетах: SynTagRus, OpenCorpora, ГИКРЯ и РНК из репозитория morphoRuEval-2017.

В основном это новостные тексты и литература. Правила razdel заточены под них.

На текстах другой тематики (социальные сети, научные статьи) библиотека может работать хуже.

In [ ]:
from razdel import sentenize
list(sentenize(example3))

https://github.com/deepmipt/ru_sentence_tokenizer
    
A simple and fast rule-based sentence segmentation. Tested on OpenCorpora and SynTagRus datasets.

In [ ]:

from rusenttokenize import ru_sent_tokenize
ru_sent_tokenize(example3)

### Бенчмарки
Много вариантов... Нужно измерять

In [ ]:
# WARNING: RAM bound task, XML parsing is expensive
# Similar to https://github.com/deepmipt/ru_sentence_tokenizer/blob/master/metrics/calculate.ipynb
import re
from lxml import etree

# \W -> Any non-word character
RE_ENDS_WITH_PUNCT = re.compile(r".*\W$")

OPENCORPORA_FILE = "annot.opcorpora.xml"
sentences = list(etree.parse(OPENCORPORA_FILE).xpath('//source/text()'))
singles = []
compounds = []
s2 = sentences.pop().strip()
singles.append(s2)
while sentences:
    s1 = sentences.pop().strip()
    singles.append(s1)
    if RE_ENDS_WITH_PUNCT.match(s1) and not s1.endswith(':') and not s2.startswith('—'):
        compounds.append((s1, s2))
    s2 = s1
        
print(f'Read {len(singles)} sentences from {OPENCORPORA_FILE}')
        
del sentences

In [ ]:
def check_sent_tokenizer(tokenizer, singles, compounds):
    correct_count_in_singles = 0
    for sentence in singles:
        correct_count_in_singles += len(tokenizer(sentence)) == 1

    correct_count_in_compounds = 0
    for s1, s2 in compounds:
        correct_count_in_compounds += tokenizer(s1 + ' ' + s2) == [s1, s2]

    return (correct_count_in_singles / len(singles), correct_count_in_compounds / len(compounds))

In [ ]:
from nltk.tokenize import sent_tokenize
%time singles_score, compounds_score = check_sent_tokenizer(sent_tokenize, singles, compounds)
print(f'sent_tokenizer scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

In [ ]:
russian_sent_tokenize = lambda s : sent_tokenize(s, language="russian")
%time singles_score, compounds_score = check_sent_tokenizer(russian_sent_tokenize, singles, compounds)
print(f'russian sent_tokenizer scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

In [ ]:
from razdel import sentenize
razdel_sent_tokenize = lambda text : [s.text for s in sentenize(text)]
%time singles_score, compounds_score = check_sent_tokenizer(razdel_sent_tokenize, singles, compounds)
print(f'razdel scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

In [ ]:
from rusenttokenize import ru_sent_tokenize
deepmipt_sent_tokenize = ru_sent_tokenize
%time singles_score, compounds_score = check_sent_tokenizer(deepmipt_sent_tokenize, singles, compounds)
print(f'deepmipt scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

Аналогичные бенчмарки:
- https://github.com/natasha/razdel/blob/master/eval.ipynb
- https://github.com/deepmipt/ru_sentence_tokenizer/blob/master/metrics/calculate.ipynb

### Задание 1: "Кирпич"
Скачайте предложенный текст. Найдите первое предложение, которое отличается в разбиениях, порождённых rusenttokenize и razdel. Верните номер этого предложения.

In [ ]:
!wget https://www.dropbox.com/s/q5wo34gfbepc7am/htbg.txt

In [ ]:
from razdel import sentenize
from rusenttokenize import ru_sent_tokenize


with open("htbg.txt", "r") as f:
    text = f.read()
    
def get_first_different_sentence(text: str) -> int:
    # YOUR CODE HERE
    return -1

assert get_first_different_sentence(text) == 329

### Задание 2: Lazy baseline
Напишите свой sent_tokenize, который будет делить предложения только по точкам, восклицательным и вопросительным знакам. Измерьте для него время работы и метрики на opencorpora.

In [ ]:
from nltk.tokenize import sent_tokenize
def my_sent_tokenize(text):
    # YOUR CODE HERE
    return []

assert my_sent_tokenize(example1) == sent_tokenize(example1)
%time singles_score, compounds_score = 0.0, 0.0 # YOUR CODE HERE
assert singles_score >= 0.85
print(f'your scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

# Токенизация

Самый наивный способ токенизировать текст -- разделить с помощью split. Но split упускает очень много всего, например, банально не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем. Поэтому лучше использовать готовые токенизаторы.

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(example5))

In [ ]:
from nltk import tokenize
dir(tokenize)[:16]

Они умеют выдавать индексы начала и конца каждого токена:

In [ ]:
from nltk import tokenize
wh_tok = tokenize.WhitespaceTokenizer()
print(list(wh_tok.span_tokenize(example5)))

Некторые токенизаторы ведут себя специфично:

In [ ]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

In [ ]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')
doc = spacy_nlp(example5, disable=["parser"])
print([token.text for token in doc])

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(example4))

In [ ]:
from razdel import tokenize
list(tokenize(example4))

### Задание 3: Diff
Напишите функцию, которая будет выводить разницу между токенизацией razdel'а и nltk.

In [ ]:
from difflib import SequenceMatcher # USE THIS
from razdel import tokenize
from nltk.tokenize import word_tokenize

with open("htbg.txt", "r") as f:
    text = f.read()
    
def get_tokenization_differences(text: str) -> int:
    differences = []
    # YOUR CODE HERE
    return differences

assert len(get_tokenization_differences(text)) == 613

# Стоп-слова и пунктуация

Стоп-слова - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конкретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

In [ ]:
from string import punctuation
punctuation

In [ ]:
noise = stopwords.words('russian') + list(punctuation)

### Задание 4: Стоп-слова from scratch
Постройте свой список стоп-слов на основе Opencorpora

In [ ]:
import re
from lxml import etree

OPENCORPORA_FILE = "annot.opcorpora.xml"
sentences = list(etree.parse(OPENCORPORA_FILE).xpath('//source/text()'))
print(f'Read {len(sentences)} sentences from {OPENCORPORA_FILE}')

# YOUR CODE HERE
 
del sentences

# Стемминг

In [ ]:
from nltk.stem.snowball import SnowballStemmer 
from razdel import tokenize

stemmer = SnowballStemmer("russian") 
print([stemmer.stem(token.text) for token in tokenize(example3)])

# Лемматизация и морфологический анализ

Лемматизация – это сведение разных форм одного слова к начальной форме – лемме. Почему это хорошо?
* Мы хотим рассматривать как отдельную фичу каждое слово, а не каждую его отдельную форму.
* Некоторые стоп-слова стоят только в начальной форме, и без лемматизации выкидываем мы только её.

Для русского есть два хороших лемматизатора: mystem и pymorphy. С pymorphy всё сразу понятно.

Но как работать с Mystem:
* Можно скачать mystem и запускать из терминала с разными параметрами
* pymystem3 - обертка для питона, работает медленнее, но это удобно

## Mystem

In [ ]:
from pymystem3 import Mystem
mystem_analyzer = Mystem()

In [ ]:
!chmod +x /root/.local/bin/mystem

Мы инициализировали Mystem c дефолтными параметрами. А вообще параметры есть такие:

    mystem_bin - путь к mystem, если их несколько
    grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
    disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
    entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

Можно просто лемматизировать текст:

In [ ]:
print(mystem_analyzer.lemmatize(example3))

А можно получить грамматическую информацию:

In [ ]:
mystem_analyzer.analyze(example3)

## Pymorphy

Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
pymorphy2_analyzer.parse("мою")

### Задание 5: Анализ частей речи

Используя pymorphy2, определите топ-10 самых частотных существительных и глаголов в тексте

In [ ]:
from pymorphy2 import MorphAnalyzer

pymorphy2_analyzer = MorphAnalyzer()
with open("htbg.txt", "r") as f:
    text = f.read()
# YOUR CODE HERE

## mystem vs. pymorphy

1) Mystem работает невероятно медленно под windows на больших текстах.

2) Снятие омонимии. Mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy2 берет на вход одно слово и соответственно вообще не умеет дизамбигуировать по контексту:


In [ ]:
homonym1 = 'За время обучения я прослушал больше сорока курсов.'
homonym2 = 'Сорока своровала блестящее украшение со стола.'
mystem_analyzer = Mystem() # инициализирую объект с дефолтными параметрами

print(mystem_analyzer.analyze(homonym1)[-5])
print(mystem_analyzer.analyze(homonym2)[0])

## Rnnmorph
Обёртка над pymorphy с разрешением омонимии

https://github.com/IlyaGusev/rnnmorph

https://habr.com/ru/post/339954/

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
from razdel import tokenize

predictor = RNNMorphPredictor(language="ru")
homonym = "Косил косой косой косой"
print(predictor.predict([t.text for t in tokenize(homonym)])[1])
print(predictor.predict([t.text for t in tokenize(homonym)])[-1])

## GramEval-2020

Соревнование по определению морфологических характеристик, определению синтаксических зависимостей и лемматизации. Готовых инструментов не получилось, но весь код всех конкурсантов доступен.
* https://github.com/dialogue-evaluation/GramEval2020

### Задание 6: Формат

Используя стандартные инструменты переведите корпус htbg.txt в формат CoNLL-U.
Используйте следующие колонки: 
    1. Номер предложения в тексте
    2. Токен в том виде, в котором он встретился в тексте
    3. Лемма токена
    4. POS-таг токена
    5. Вектор грамматических значений токена
    6. Целевая метка (сделайте метку везде OUT)

# Regex 101

In [ ]:
import re

#### match
ищет по заданному шаблону в начале строки

In [ ]:
result = re.match('ab+c.', 'abcdefghijkabcabc') # ищем по шаблону 'ab+c.' 
print (result) # совпадение найдено:

In [ ]:
print(result.group(0)) # выводим найденное совпадение

In [ ]:
result = re.match('abc.', 'abdefghijkabcabc')
print(result) # совпадение не найдено

#### search
ищет по всей строке, возвращает только первое найденное совпадение

In [ ]:
result = re.search('ab+c.', 'aefgabchijkabcabc') 
print(result) 

#### findall
возвращает список всех найденных совпадений

In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

Вопросы: 
1) почему нет последнего abc?
2) почему нет abcx?

#### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit = 2) 
print(result)

#### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

#### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

In [ ]:
# Ваш код